# Ingest data into elastic search cluster

In this notebook we first download the raw data from the dataset server and then ingest into an elasticsearch cluster.

1. The configuration parameters are read from a local config file ingest.cfg.

2. To test whether the data is correctly inserted we also run a query on the inserted data.

In [10]:
# Copyright (C) 2018, Hughes Network Systems, LLC - All Rights Reserved
# Unauthorized copying of this file, via any medium is strictly prohibited
# Proprietary and confidential
# Written by Amit Arora <amit.arora@hughes.com>, December 2018

## Global imports and utility functions

In [11]:
# global imports
import os
import sys
import json
import logging
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from configparser import ConfigParser
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import seaborn as sns
sns.set()

# check if the elasticsearch package is installed, if not give an error message that we cannot continue
try:
    from elasticsearch import Elasticsearch
    from elasticsearch.helpers import bulk
except ImportError as e:
    print("package elasticsearch is not installed, cannot continue without it. Install using pip install elasticsearch")
    
# constants
DEFAULT_CFG_FILE_NAME = "/var/tmp/nlp/nlp.cfg"
NUM_RANDOM_ROWS = 5
pd.set_option('display.max_columns', None)

'''
Start with a simple console logger until we process the configuration. Then
we will reconfigure the logging as configured. This at least gives some console
logging while processing configuration.
'''
logging.basicConfig(
    format='%(asctime)s,%(levelname)s,%(module)s,%(message)s',
    level=logging.INFO,
    stream=sys.stdout)

logger = logging.getLogger(__name__)

In [12]:
def read_config(cfg_file_path):
    '''
    Read config file /var/tmp/nlp/nlp.cfg if it exists, returns the content
    as a dictionary. Empty dictionary is returned if file
    is not present.
    Sample file:


[Elasticsearch]
host: vmwhnsdscemon01.nadops.net
port: 9200
index: csat-tier1

[App]
name: nomadds_nlp
version: 1.0.0.0

[Dataset]
server: dscapp.hnops.net
version: 1.0.0.0
raw_data_dir: raw_data
filenames: CSAT_Tier_1_Sutherland_Philippines_v2_October_26_2018.csv,CSAT_Tier 1_Sutherland_Legazpi_v2_October_26_2018.csv,CSAT_Tier_1_Sutherland_Tarlac_October_26_2018.csv,CSAT_Tier_1_TP_Brownsville_v2_October_26_2018.csv,CSAT_Tier_1_TP_Monterrey_v2_October_26_2018.csv
rows_to_skip: 1,2

[Output]
data_dir: data
figures_dir: figures

    '''
    # check if file exists
    if Path(cfg_file_path).is_file() is False:
        logger.error("config file {} does not exist, cannot continue...".format(cfg_file_path))
        return {}

    # empty dictionary to be filled as we read the section
    # in the config file
    config_dict = {}
    config = ConfigParser()
    logger.info("going to read config from {}".format(cfg_file_path))
    config.read(cfg_file_path)
    sections = config.sections()
    for section in sections:
        if section not in config_dict.keys():
            config_dict[section] = {}
        params_dict = dict(config.items(section))
        for param_name in params_dict.keys():
            config_dict[section][param_name] = params_dict[param_name]

    # modify some of the parameters to fix types
    # also some additional massaging like converting a comma separate list to
    # to a python list
    try:
        config_dict['Elasticsearch']['port'] = config.get('Elasticsearch', 'port', fallback="9200")
        config_dict['Elasticsearch']['max_resp_count'] = config.getint('Elasticsearch', 'max_resp_count', fallback=10000)
        config_dict['Elasticsearch']['delete_index_if_present'] = config.getboolean('Elasticsearch', 'delete_index_if_present', fallback=False)
        
        config_dict['Dataset']['filenames'] = config_dict['Dataset']['filenames'].split(',')
        config_dict['Dataset']['rows_to_skip'] = [int(r) for r in config_dict['Dataset']['rows_to_skip'].split(',')]
        
        config_dict['Dataset']['columns_of_interest'] = config.get('Dataset', 'columns_of_interest', fallback=None)
        config_dict['Dataset']['columns_of_interest'] = config_dict['Dataset']['columns_of_interest'].split(',')
        
        config_dict['Dataset']['fields_for_barplot'] = config.get('Dataset', 'fields_for_barplot', fallback=None)
        config_dict['Dataset']['fields_for_barplot'] = config_dict['Dataset']['fields_for_barplot'].split(',')
        
        config_dict['Dataset']['replace_na_bool'] = config.getboolean('Dataset', 'replace_na_bool', fallback=False)
        config_dict['Dataset']['replace_na_float'] = config.getfloat('Dataset', 'replace_na_float', fallback=-1)
        config_dict['Dataset']['replace_na_object'] = config.get('Dataset', 'replace_na_object', fallback="unknown")
        
        config_dict['Dataset']['q_field'] = config.get('Dataset', 'q_field', fallback="unknown")
        config_dict['Dataset']['q_value'] = config.get('Dataset', 'q_value', fallback="unknown")
        
        config_dict['Output']['base_dir'] = config.get('Output', 'base_dir', fallback="/var/tmp/nlp")
        
        config_dict['NER']['entity_counts_file_name'] = config.get('NER', 'entity_counts_file_name', fallback="ner.csv")
        config_dict['NER']['entity_counts_timeseries_file_name'] = config.get('NER', 'entity_counts_timeseries_file_name',
                                                                              fallback="ner_yearmon.csv")  
        
        config_dict['NER']['variants'] = config.get('NER', 'variants', fallback=None)
        config_dict['NER']['variants'] = config_dict['NER']['variants'].split(',')
        config_dict['NER']['variants']  = [(x.split("__")[0], x.split("__")[1:]) for x in config_dict['NER']['variants']]
        
        config_dict['NER']['name_entity_mapping'] = config.get('NER', 'name_entity_mapping', fallback=None)
        config_dict['NER']['name_entity_mapping'] = config_dict['NER']['name_entity_mapping'].split(',')
        config_dict['NER']['name_entity_mapping']  = [(x.split("__")[0], x.split("__")[1]) for x in config_dict['NER']['name_entity_mapping']]
        
        config_dict['NER']['stop_words'] = config.get('NER', 'stop_words', fallback=None)
        config_dict['NER']['stop_words'] = config_dict['NER']['stop_words'].split(',')
        
        config_dict['NER']['bar_plot_threshold'] = config.getint('NER', 'bar_plot_threshold', fallback=DEFAULT_BAR_PLOT_THRESHOLD)
        
        config_dict['NER']['fields_for_barplot'] = config.get('NER', 'fields_for_barplot', fallback='ORG')
        config_dict['NER']['fields_for_barplot'] = config_dict['NER']['fields_for_barplot'].split(',')
        
        config_dict['TopicModeling']['num_topics'] = config.getint('TopicModeling', 'num_topics', fallback=10)
        config_dict['TopicModeling']['imp_words_per_topic'] = config.getint('TopicModeling', 'imp_words_per_topic', fallback=10)
        config_dict['TopicModeling']['lda_model_random_seed'] = config.getint('TopicModeling', 'lda_model_random_seed', fallback=1603)
        config_dict['TopicModeling']['lda_model_num_passes'] = config.getint('TopicModeling', 'lda_model_num_passes', fallback=3)
        
        config_dict['TopicModeling']['topics'] = config.get('TopicModeling', 'topics', fallback=None)
        config_dict['TopicModeling']['topics'] = config_dict['TopicModeling']['topics'].split(',')
        config_dict['TopicModeling']['topics'] = [x.replace("\"", "") for x in config_dict['TopicModeling']['topics']]
        config_dict['TopicModeling']['topics'] = [x.replace(";", ",") for x in config_dict['TopicModeling']['topics']]
        config_dict['TopicModeling']['num_topics'] = len(config_dict['TopicModeling']['topics'])
        
        config_dict['SentimentAnalysis']['category1'] = config.get('SentimentAnalysis', 'category1', fallback=None)
        config_dict['SentimentAnalysis']['category1'] = config_dict['SentimentAnalysis']['category1'].split(',')
        config_dict['SentimentAnalysis']['category1_label'] = config_dict['SentimentAnalysis']['category1'][0]
        config_dict['SentimentAnalysis']['category1_leftedge'] = int(config_dict['SentimentAnalysis']['category1'][1])
        config_dict['SentimentAnalysis']['category1_rightedge'] = int(config_dict['SentimentAnalysis']['category1'][2])
        
        config_dict['SentimentAnalysis']['category2'] = config.get('SentimentAnalysis', 'category2', fallback=None)
        config_dict['SentimentAnalysis']['category2'] = config_dict['SentimentAnalysis']['category2'].split(',')
        config_dict['SentimentAnalysis']['category2_label'] = config_dict['SentimentAnalysis']['category2'][0]
        config_dict['SentimentAnalysis']['category2_leftedge'] = int(config_dict['SentimentAnalysis']['category2'][1])
        config_dict['SentimentAnalysis']['category2_rightedge'] = int(config_dict['SentimentAnalysis']['category2'][2])
        
        config_dict['SentimentAnalysis']['learning_rate'] = config.getfloat('SentimentAnalysis', 'learning_rate', fallback=0.01)
        config_dict['SentimentAnalysis']['decay'] = config.getfloat('SentimentAnalysis', 'decay', fallback=0.000001)
        config_dict['SentimentAnalysis']['momentum'] = config.getfloat('SentimentAnalysis', 'momentum', fallback=0.9)
        config_dict['SentimentAnalysis']['nesterov'] = config.getboolean('SentimentAnalysis', 'nesterov', fallback=True)
        config_dict['SentimentAnalysis']['deeplearning_model'] = config.get('SentimentAnalysis', 'deeplearning_model', fallback='mlp')
        config_dict['SentimentAnalysis']['epochs'] = config.getint('SentimentAnalysis', 'epochs', fallback=150)
        config_dict['SentimentAnalysis']['batch_size'] = config.getint('SentimentAnalysis', 'batch_size', fallback=1024)
        config_dict['SentimentAnalysis']['validation_split'] = config.getfloat('SentimentAnalysis', 'validation_split', fallback=0.2)
        config_dict['SentimentAnalysis']['test_split'] = config.getfloat('SentimentAnalysis', 'test_split', fallback=0.2)
        config_dict['SentimentAnalysis']['model_file_name'] = config.get('SentimentAnalysis', 'model_file_name', fallback="sentiment_analysis.h5")
        config_dict['SentimentAnalysis']['model_metrics_file_name'] = config.get('SentimentAnalysis', 'model_metrics_file_name', fallback="model_metrics.csv")
        config_dict['SentimentAnalysis']['model_accuracy_plot_file_name'] = config.get('SentimentAnalysis', 'model_accuracy_plot_file_name', fallback="model_accuracy_plot.png")
        config_dict['SentimentAnalysis']['model_loss_plot_file_name'] = config.get('SentimentAnalysis', 'model_loss_plot_file_name', fallback="model_loss_plot.png")
        config_dict['SentimentAnalysis']['model_cfm_plot_file_name'] = config.get('SentimentAnalysis', 'model_cfm_plot_file_name', fallback="model_cfm_plot.png")
       
    except Exception as e:
        logger.error("exception while adjusting data type of configuration parameters")
        logger.error(str(e))

    logger.info("config dictionary")
    logger.info(json.dumps(config_dict, indent=2))
    return config_dict

In [15]:
# utility function for creating a file path from filename
def get_output_file_path(config, filename):
    dir_path = os.path.join(config['Output']['base_dir'], config['Output']['data_dir'], config['Dataset']['version'])
    os.makedirs(dir_path, exist_ok=True)
    return os.path.join(dir_path, filename)

def get_figures_file_path(config, filename):
    dir_path = os.path.join(config['Output']['base_dir'], config['Output']['figures_dir'], config['Dataset']['version'])
    os.makedirs(dir_path, exist_ok=True)
    return os.path.join(dir_path, filename)

## Read the configuration

In [14]:
# read the configuration
config = read_config(DEFAULT_CFG_FILE_NAME)

2019-01-02 02:48:32,116,INFO,<ipython-input-12-473443e7ee40>,going to read config from /var/tmp/nlp/nlp.cfg
2019-01-02 02:48:32,120,INFO,<ipython-input-12-473443e7ee40>,config dictionary
2019-01-02 02:48:32,122,INFO,<ipython-input-12-473443e7ee40>,{
  "Elasticsearch": {
    "host": "http://vmwhnsdscemon01.nadops.net",
    "port": "9200",
    "index": "csat-tier1",
    "doc_type": "tier1-survey",
    "max_resp_count": 10000,
    "delete_index_if_present": false
  },
  "App": {
    "name": "nomadds_nlp",
    "version": "1.0.0.0"
  },
  "Dataset": {
    "name": "csat",
    "server": "http://dscapp.hnops.net",
    "server_root": "datasets",
    "version": "1.0.0.0",
    "raw_data_dir": "raw_data",
    "filenames": [
      "CSAT_Tier_1_Sutherland_Philippines_v2_October_26_2018.csv",
      "CSAT_Tier 1_Sutherland_Legazpi_v2_October_26_2018.csv",
      "CSAT_Tier_1_Sutherland_Tarlac_October_26_2018.csv",
      "CSAT_Tier_1_TP_Brownsville_v2_October_26_2018.csv",
      "CSAT_Tier_1_TP_Monterre

## Download the raw data from the dataset server

In [5]:
dataset_urls = list(map(lambda x: "/".join([config['Dataset']['server'], 
                                            config['Dataset']['server_root'],
                                            config['Dataset']['name'], 
                                            config['Dataset']['version'],
                                            config['Dataset']['raw_data_dir'], x]),
                        config['Dataset']['filenames']))
dataset = pd.concat(list(map(lambda url: pd.read_csv(url, skiprows=config['Dataset']['rows_to_skip']), dataset_urls)))
logging.info("shape of the raw dataset is {} ".format(dataset.shape))
logging.info("printing {} random rows from the dataset".format(NUM_RANDOM_ROWS))
display(dataset.sample(NUM_RANDOM_ROWS))

/opt/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (9,10,15,16,33,37,39,40,41,42,44,45,46,47,48,49,50,51,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,70,71,72,73,74,75,76,77,78,79,80,81,83,84,85,86,87,88,89,90,91,92,93,94,95,98,99,100,101,102,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (9,10,15,16,33,40,41,42,44,45,46,47,48,49,50,51,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,70,71,72,73,74,75,76,77,78,79,80,81,83,84,85,86,87,88,89,90,91,92,93,94,95,98,99,100,101,102,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (9,10,33,40,41,42,44,45,46,47,48,50,51,53,54,55,56,57,58,59,60,61,62,63,64,6

2018-12-30 20:15:44,313,INFO,<ipython-input-5-06775b58c0fb>,shape of the raw dataset is (446940, 111) 
2018-12-30 20:15:44,315,INFO,<ipython-input-5-06775b58c0fb>,printing 5 random rows from the dataset


,AccessGateway,AdapterModel,Address,Agent,CaseID,CaseOpenDate,CaseSource,CaseType,City,CompanyType,DistributionChannel,DownlinkCell,Duration (in seconds),EndDate,ExternalReference,Finished,FirstCommissionDate,HardwareProgram,IPAddress,InvitationDate,LocationLatitude,LocationLongitude,Phone,Postal,ProblemArea,ProblemDetail,ProblemType,Progress,ProviderGroup,Q11,Q12,Q14,Q15,Q16,Q17,Q18,Q21,Q2_Browser,Q2_Operating System,Q2_Resolution,Q2_Version,Q3,Q4,Q4_4_TEXT,Q5_1,Q5_2,Q5_3,Q5_4,Q5_5,Q5_6,Q5_Q5_1,Q5_Q5_2,Q5_Q5_3,Q5_Q5_4,Q5_Q5_5,Q5_Q5_6,Q6_1,Q6_2,Q6_3,Q7,Q8,Q9,RecipientEmail,RecipientFirstName,RecipientLastName,RecordedDate,ResponseId,SAN,SalesChannel,SalesSource,ServicePlan,SiteType,StartDate,State,Status,SurveyName,Transponder,UplinkCell,UserLanguage,sfAccess_Gateway__c,sfAccountNumber__c,sfAdapter_Model__c,sfAddress__c,sfAgent__c,sfCaseNumber__c,sfCaseOpenTimeinCustomerTimeZone__c,sfCase_Source__c,sfCase_SubType__c,sfCase_Type__c,sfCity__c,sfCompany_Type__c,sfContact_First_Name__c,sfContact_Last_Name__c,sfDownlinkCell__c,sfFirst_Commission_Date__c,sfHardware_Program__c,sfInvitation_Date__c,sfName,sfPhone__c,sfPostal_Code__c,sfProblem_Area__c,sfProblem_Detail__c,sfProblem_Type__c,sfProvider_Group__c,sfSales_Channel__c,sfSales_Source__c,sfService_Plan__c,sfSite_Type__c,sfState__c,sfTransponder__c,sfUplinkCell__c
39301,NaN,HT1000,307 Overlook Dr,tp-cmontelongo,44565836.0,2014-11-04,PHONE,Technical,Vicksburg,NaN,NaN,NaN,159.0,2014-11-05 14:46:59,DSS30610377,True,02/28/2013 00:00:00,Lease,97.73.51.56,2014-11-05,35.880707,-82.309502,NaN,39180,Update Payment Information,Payment Details,Account,NaN,TPBRWN,10 - Very Likely,NaN,8,NaN,NaN,NaN,NaN,NaN,Mozilla,Windows NT 6.3,2077x1168,5.0,Yes,Billing Issue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,Yes,No,No,NaN,Yes,Yes,jonjonbaileigh@gmail.com,johnnie,grantham,2014-11-05 14:46:59,R_8iT9NsiRbEE0RUx,DSS30610377,Direct,OKS Ameridial Inc (CC),NaN,Jupiter,2014-11-05 14:44:20,MS,IP Address,TPC - Customer Support Experience v1.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,email,NaN,221.0,2018-10-15 11:20:03,04l0a00001FEsYtAAL,True,NaN,NaN,174.231.168.111,NaN,38.250305,-85.703400,NaN,NaN,NaN,NaN,NaN,100.0,NaN,10 - Very Likely,NaN,10 - Very Satisfied,NaN,NaN,NaN,NaN,Completely agree,Safari iPhone,iPhone,320x568,12.0,Yes,Other,NaN,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,NaN,NaN,NaN,NaN,NaN,NaN,No,No,No,NaN,Yes,Yes,robinson3126@bellsouth.net,NaN,NaN,2018-10-15 11:20:03,R_ugL4VcpizLVqhep,NaN,NaN,NaN,NaN,NaN,2018-10-15 11:16:22,NaN,IP Address,NaN,NaN,NaN,EN,NaN,DSS35672536,HT2000W,47 TURKEY CRK,sutr-eicastro@sutherlandglobal.com,116049145.0,10/15/2018 11:09AM,Phone,Information,Non-Technical,LANGLEY,Jupiter,Sandra,Robinson,NaN,2017-12-09,Lease,2018-10-15,CSAT Tier 1 - SUT Tarlac,(606) 285-5080,41645,Update Payment Info,Resolved,Billing Info,SUT Tarlac Universal,Direct,Communication Solutions LLC (CC),Gen5_20GB,Jupiter,KY,NaN,NaN
37129,NaN,HT1100,472 2nd St,sutb-irishdavid,44753011.0,2014-11-15,IVR,Technical,Plymouth,NaN,NaN,NaN,318.0,2014-11-16 11:21:59,DSS32186948,True,07/16/2014 00:00:00,Purchase,69.35.195.55,2014-11-16,38.000000,-97.000000,NaN,18651,General Information,Usage,Account,NaN,SUTBUA,1- Very Unlikely,same as above,1 - Very Dissatisfied,find a resolution to our problem. stop doing t...,NaN,NaN,NaN,NaN,Mozilla,Windows NT 6.3,1366x768,5.0,Yes,Other,data disappearing,3,3,1 - Very Dissatisfied,1 - Very Dissatisfied,1 - Very Dissatisfied,1 - Very Dissatisfied,NaN,NaN,NaN,NaN,NaN,NaN,Yes,No,No,NaN,No,NaN,passfounder@yahoo.com,Joseph and Monica,Thomas,2014-11-16 11:21:59,R_ekP51KMb7amebd3,DSS32186948,Direct,Red Ventures - Web,NaN,Jupiter,2014-11-16 11:16:41,PA,IP Address,TPC 

## Data cleaning
Some minimal data cleaning is necessary before inserting the data into elastic search. For example, we need to replace all NA/NaN with data type specific values before the data can be inserted into elastcisearch.
Also some fields are difficult to fix this way, such as a date field. These fields should be excluded from the columns_of_interest config parameter.

In [6]:
# we only want to keep columns of interest, if configured
if config['Dataset']['columns_of_interest'] is not None:
    dataset = dataset[config['Dataset']['columns_of_interest']]
    logging.info("shape of the dataset after only keeping columns of interest is {} ".format(dataset.shape))
    logging.info("printing {} random rows from the subsetted dataset".format(NUM_RANDOM_ROWS))
    display(dataset.sample(NUM_RANDOM_ROWS))

# replace NA with None
sink = [fill_dtype_based_na(dataset, c) for c in dataset.columns]
logger.info("after replacing NA values (if any in the dataset)")
display(dataset.sample(NUM_RANDOM_ROWS))

2018-12-30 20:15:46,118,INFO,<ipython-input-6-3f821a3843fe>,shape of the dataset after only keeping columns of interest is (446940, 77) 
2018-12-30 20:15:46,119,INFO,<ipython-input-6-3f821a3843fe>,printing 5 random rows from the subsetted dataset


,AccessGateway,AdapterModel,Address,Agent,CaseSource,CaseType,City,CompanyType,DistributionChannel,DownlinkCell,Duration (in seconds),EndDate,ExternalReference,Finished,FirstCommissionDate,HardwareProgram,IPAddress,LocationLatitude,LocationLongitude,Phone,Postal,ProblemArea,ProblemDetail,ProblemType,Progress,ProviderGroup,Q11,Q12,Q14,Q15,Q16,Q17,Q18,Q21,Q2_Browser,Q2_Operating System,Q2_Resolution,Q2_Version,Q3,Q4,Q4_4_TEXT,Q5_1,Q5_2,Q5_3,Q5_4,Q5_5,Q5_6,Q5_Q5_1,Q5_Q5_2,Q5_Q5_3,Q5_Q5_4,Q5_Q5_5,Q5_Q5_6,Q6_1,Q6_2,Q6_3,Q7,Q8,Q9,RecipientEmail,RecipientFirstName,RecipientLastName,RecordedDate,ResponseId,SAN,SalesChannel,SalesSource,ServicePlan,SiteType,StartDate,State,Status,SurveyName,Transponder,UplinkCell,UserLanguage,sfSite_Type__c
99802,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,email,NaN,271.0,2018-06-22 19:03:41,04l0a000017bjsTAAQ,True,NaN,NaN,67.142.112.206,37.751007,-97.821999,NaN,NaN,NaN,NaN,NaN,100.0,NaN,9,Don't have any more feedback,5,NaN,NaN,NaN,NaN,NaN,Safari iPhone,iPhone,320x568,10.0,Yes,Billing Issue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,No,Yes,No,NaN,Yes,Yes,jhjrgoss@yahoo.com,NaN,NaN,2018-06-22 19:03:41,R_3Gkwc4KDSwZaBRn,NaN,NaN,NaN,NaN,NaN,2018-06-22 18:59:09,NaN,IP Address,NaN,NaN,NaN,EN,Jupiter
141188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,email,NaN,42.0,2018-08-09 20:46:09,04l0a00001AVFh0AAH,True,NaN,NaN,67.143.129.123,37.751007,-97.821999,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chrome,Android 5.1.1,600x1024,67.0.3396.87,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cwilliec48@aol.com,NaN,NaN,2018-08-09 20:46:09,R_3luexGiyfhF9OYE,NaN,NaN,NaN,NaN,NaN,2018-08-09 20:45:26,NaN,IP Address,NaN,NaN,NaN,EN,Jupiter
52818,NaN,HT1000,1335 Gamble Rd,sutb-malmallari,PHONE,Billing,New Florence,NaN,email,NaN,89.0,2015-11-19 04:29:59,DSS30365354,True,11/18/2012 00:00:00,Lease,173.209.211.239,38.000000,-97.000000,NaN,15944,Payment,Full Payment,Billing,100.0,SUTBUA,10 - Very Likely,NaN,10 - Very Satisfied,NaN,NaN,NaN,NaN,NaN,Chrome,Android 4.4.4,320x534,45.0.2454.84,Yes,Billing Issue,NaN,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,NaN,NaN,NaN,NaN,NaN,NaN,No,No,No,NaN,Yes,Yes,allencwood@gmail.com,Craig,Woodward,2015-11-19 04:29:59,R_eX1bEhUTfs5gASN,DSS30365354,Direct,Red Ventures - Web,NaN,Jupiter,2015-11-19 04:28:30,PA,IP Address,TPC - Customer Support Experience v1.1,NaN,NaN,NaN,NaN
45967,NORNE1067AGWHNS00806,HN9000,13658 County Road 138 W,sutb-mlacap,PHONE,Technical,Vernon,NaN,email,210.0,208.0,2015-06-19 06:40:41,DSS30062592,True,05/17/2012 00:00:00,Lease,67.142.167.24,38.000000,-97.000000,NaN,76384,General Information,Service Plans,Account,100.0,SUTBUA,10 - Very Likely,Very polite and helpful - spent a lot of time ...,10 - Very Satisfied,NaN,NaN,NaN,NaN,NaN,Safari iPad,iPad,768x1024,5.1,Yes,Service Performance/Outage Issues,NaN,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,NaN,NaN,NaN,NaN,NaN,NaN,Yes,No,Yes,Yes,Yes,Yes,japstreit@yahoo.com,JAMES,STREIT,2015-06-19 06:40:41,R_aVJgSVStZONEvmR,DSS30062592,Hughes Sales Agents,Red Ventures LLC (SA),NaN,Spaceway,2015-06-19 06:37:13,TX,IP Address,TPC - Customer Support Experience v1.1,NaN,23.0,NaN,NaN
18223,NaN,HT1100,5166 W 700 N,tpm-acruzamen,IVR,Inquiry,Delphi,NaN,NaN,NaN,624.0,2014-12-24 16:28:53,DSS32463200,True,11/03/2014 00:00:00,Lease,69.35.194.123,39.152695,-83.039101,NaN,46923,Explanation of Charges,Service Plan Billing,Billing,NaN,TPMON,10 - Very Likely,right now everything is great.thank you and me...,10 - Very Satisfied,NaN,NaN,NaN,NaN,NaN,MSIE,Windows NT 6.0,1280x1024,9.0,Yes,Billing Issue,NaN,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,NaN,NaN,NaN,NaN,NaN,NaN,Yes,No,No,NaN,Yes,Yes,pattyhammel31@y

2018-12-30 20:15:49,944,INFO,<ipython-input-6-3f821a3843fe>,after replacing NA values (if any in the dataset)


,AccessGateway,AdapterModel,Address,Agent,CaseSource,CaseType,City,CompanyType,DistributionChannel,DownlinkCell,Duration (in seconds),EndDate,ExternalReference,Finished,FirstCommissionDate,HardwareProgram,IPAddress,LocationLatitude,LocationLongitude,Phone,Postal,ProblemArea,ProblemDetail,ProblemType,Progress,ProviderGroup,Q11,Q12,Q14,Q15,Q16,Q17,Q18,Q21,Q2_Browser,Q2_Operating System,Q2_Resolution,Q2_Version,Q3,Q4,Q4_4_TEXT,Q5_1,Q5_2,Q5_3,Q5_4,Q5_5,Q5_6,Q5_Q5_1,Q5_Q5_2,Q5_Q5_3,Q5_Q5_4,Q5_Q5_5,Q5_Q5_6,Q6_1,Q6_2,Q6_3,Q7,Q8,Q9,RecipientEmail,RecipientFirstName,RecipientLastName,RecordedDate,ResponseId,SAN,SalesChannel,SalesSource,ServicePlan,SiteType,StartDate,State,Status,SurveyName,Transponder,UplinkCell,UserLanguage,sfSite_Type__c
61622,unknown,HT1100,1619 Price House Rd,sutb-rebuen,CHAT,Billing,Roebuck,unknown,email,-1.0,242.0,2016-12-20 11:09:59,DSS34457981,True,12/16/2016 00:00:00,Lease,104.129.204.74,33.748993,-84.388000,864 5048358,29376,Statement,Online Access,Billing,100.0,SUTBUA,8,Instead of 50 bonus gb data between 2am and 8a...,8,unknown,unknown,unknown,unknown,unknown,Chrome,Android 6.0.1,360x640,55.0.2883.91,Yes,Billing Issue,unknown,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,unknown,10 - Very Satisfied,unknown,unknown,unknown,unknown,unknown,unknown,Yes,No,No,unknown,Yes,Yes,thatkaratekid@yahoo.com,Daniel,Brown,2016-12-20 11:09:59,R_3Ra5csyPB55pMS9,DSS34457981,Direct,Red Ventures - Web,unknown,Jupiter,2016-12-20 11:05:57,SC,IP Address,TPC - Customer Support Experience v1.1,unknown,-1.0,unknown,unknown
78286,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,email,-1.0,34.0,2018-07-30 10:53:51,04l0a000019oxoxAAA,True,unknown,unknown,174.255.6.25,37.751007,-97.821999,unknown,unknown,unknown,unknown,unknown,100.0,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,Chrome,Android 5.1.1,360x640,68.0.3440.70,No,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,aliyahmiel98@gmail.com,unknown,unknown,2018-07-30 10:53:52,R_3J4UGKxKNcpQlu3,unknown,unknown,unknown,unknown,unknown,2018-07-30 10:53:17,unknown,IP Address,unknown,unknown,-1.0,EN,Jupiter
33789,AMATX1036AGWHNS00801,HN9000,8574 S 60th Rd,sutb-dgardingan,PHONE,Billing,Alda,unknown,unknown,574.0,179.0,2014-08-02 18:47:52,DSS30456806,True,12/28/2012 00:00:00,Lease,67.142.170.21,38.000000,-97.000000,unknown,68810,Update Payment Information,Payment Details,Account,-1.0,SUTBUA,10 - Very Likely,unknown,10 - Very Satisfied,unknown,unknown,unknown,unknown,unknown,Firefox,Windows NT 6.1,1600x900,30.0,Yes,Billing Issue,unknown,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,unknown,unknown,unknown,unknown,unknown,unknown,No,No,No,unknown,Yes,Yes,garyr0916@hamilton.net,gary,rezac,2014-08-02 18:47:52,R_5o1l1yOPkwFndDT,DSS30456806,Direct,ACS,unknown,Spaceway,2014-08-02 18:44:53,NE,IP Address,TPC - Customer Support Experience v1.1,unknown,68.0,unknown,unknown
12669,unknown,HT1100,4414 Ford St,sutl-jaliven,IVR,Technical,Gulfport,unknown,unknown,-1.0,271.0,2014-04-14 04:48:56,DSS31941678,True,04/08/2014 00:00:00,Lease,166.147.120.47,38.000000,-97.000000,unknown,39501,Browsing,No Browse,Performance,-1.0,SUTL,6,Provide wireless router,6,unknown,unknown,unknown,unknown,unknown,Safari iPhone,iPhone,320x480,6.0,Yes,Question/Inquiry,unknown,unknown,unknown,unknown,unknown,unknown,unknown,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,10 - Very Satisfied,Yes,No,No,unknown,Yes,Yes,trishandjimmy23@gmail.com,JAMES,HUTCHISON,2014-04-14 04:48:56,R_9Lxh8KINCLlNo7X,DSS31941678,Hughes Sales Agents,Bundlefinders.com LLC (SA),unknown,Jupiter,2014-04-14 04:44:25,MS,IP Address,TPC - Customer Support Experience v1.1,unknown,-1.0,unknown,unknown
25434,unknown,HT1100,3133 Wilkens Ave,sutl-bmaranda,IVR,Te

## Basic visualizations

In [16]:
# if any fields are configured for barplots then plot then
def plot_bar_chart(field, config):
    plt.figure(figsize=(16, 6))
    sns.countplot(x=field, data=dataset)
    plt.savefig(get_figures_file_path(config, field + "_counts.png"))
    
if config['Dataset']['fields_for_barplot'] is not None:  
    plots = list(map(lambda field: plot_bar_chart(field, config), config['Dataset']['fields_for_barplot']))

NameError: name 'dataset' is not defined

<Figure size 1152x432 with 0 Axes>

## Ingest data into elasticsearch
See https://creativedata.atlassian.net/wiki/spaces/SAP/pages/130252820/Python+-+Index+Search+documents+in+ElasticSearch for code reference. If there are any errors during ingestion in elasticsearch then the ingestion would stop, this would require either removing the offending columns from the columns of interest configuration parameter or a change in the code to resolve the error.

In [8]:
# ====== Connection ====== #
# Connection to ElasticSearch
es = Elasticsearch([config['Elasticsearch']['host'] +':'+ config['Elasticsearch']['port']],timeout=600)

# Simple index creation with no particular mapping
ignore_errors = [400, 404]
es.indices.delete(index=config['Elasticsearch']['index'], ignore=ignore_errors)
es.indices.create(index=config['Elasticsearch']['index'], body={})

2018-12-30 20:15:52,097,INFO,base,DELETE http://vmwhnsdscemon01.nadops.net:9200/csat-tier1 [status:200 request:0.790s]
2018-12-30 20:15:52,212,INFO,base,PUT http://vmwhnsdscemon01.nadops.net:9200/csat-tier1 [status:200 request:0.113s]


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'csat-tier1'}

In [9]:
# ====== Inserting Documents ====== #
# Bulk inserting documents. Each row in the DataFrame will be a document in ElasticSearch
start = datetime.now()
documents = dataset.to_dict(orient='records')
logger.info("took {} to convert {} documents to json".format(datetime.now()-start, len(documents)))

2018-12-30 20:16:34,502,INFO,<ipython-input-9-4b84888f8c2d>,took 0:00:42.277893 to convert 446940 documents to json


In [10]:
start = datetime.now()
# change the log level because elastcisearch client prints a message for ever N (typically 500 records)
logging.getLogger().setLevel(logging.ERROR)
bulk(es, documents, index=config['Elasticsearch']['index'], doc_type=config['Elasticsearch']['doc_type'], raise_on_error=True, raise_on_exception=True)
logging.getLogger().setLevel(logging.INFO)
logger.info("took {} to insert {} documents".format(datetime.now()-start, len(documents)))

2018-12-30 20:18:56,447,INFO,<ipython-input-10-b97e467e1e92>,took 0:02:21.936597 to insert 446940 documents


## Search the ingested data

In [78]:
# ====== Searching Documents ====== #
# Retrieving documents in index that match a query
documents = es.search(index=config['Elasticsearch']['index'],
                      body={"size":config['Elasticsearch']['max_resp_count'],
                            "query":{"match":{config['Dataset']['q_field'] : config['Dataset']['q_value'] }}})['hits']['hits']
df = pd.DataFrame(documents)

# what we have at this point is a dataframe which contains a field called "_source" which 
# actually contains the response data. The data in _source is in the form of a dictionary
# so we need to confirm each value in the source column into a dataframe and then concat
# all of them to finally get a dataframe which is of the format that we were expecting
df = pd.concat([pd.DataFrame.from_dict(x, orient='index').T for x in df['_source'].tolist()])
df = df.reset_index(drop=True)
logger.info("searching for \"{}\" in field \"{}\" returend {} matching documents".format(config['Dataset']['q_value'],
                                                                                 config['Dataset']['q_field'],
                                                                                len(df)))
display(df[config['Dataset']['q_field']])

2018-12-31 05:18:39,649,INFO,base,GET http://vmwhnsdscemon01.nadops.net:9200/csat-tier1/_search [status:200 request:0.063s]
2018-12-31 05:18:40,957,INFO,<ipython-input-78-923b55aa8369>,searching for "video" in field "Q12" returend 1781 matching documents


0            Streaming Video stops while viewing a video.
1                                 Provide streaming video
2                                     Faster video feeds.
3                                     Cannot Stream Video
4                           Needs better streaming video 
5                           Offer faster streaming video.
6                               Streaming video is slow. 
7       Service is slow.  Downloading video clips take...
8                     Faster speed better video streaming
9                 Video streaming has many interruptions.
10                Continuity when viewing video downloads
11               Slow speed.  Can't stream video \r\n\r\n
12                      improve service for video calling
13            Faster internet and video streaming\r\n\r\n
14                            Video download is very slow
15                         Buffering when you watch video
16                   Streaming video without data charges
17            